In [ ]:
using PyPlot
pygui(true)

function load_matrix(filename)
    open(filename) do io
        c = read(io, Char)
        @assert c == ','
        line = readuntil(io, '\n')
        ns = parse.(Int, split(line, ','))
        lines = readlines(io)
        mlen = length(lines)
        tqs = Matrix{Float64}(undef, mlen, length(ns))
        tls = similar(tqs)
        ms = Vector{Int}(undef, mlen)
        for (i, line) in enumerate(lines)
            m = match(r"(\d+),", line)
            ms[i] = parse(Int, m.captures[1])
            idx = m.offset + length(m.captures[1])
            for j = 1:length(ns)
                ts = match(r"\(([+-]?[0-9]*[.]?[0-9e+-]+),([+-]?[0-9]*[.]?[0-9e+-]+)\)", line, idx)
                tqs[i,j] = parse(Float64, ts.captures[1])
                tls[i,j] = parse(Float64, ts.captures[2])
                idx += length(ts.match)
            end
        end
        return ms, ns, tqs, tls
    end
end

ms_jl, ns_jl, tqs_jl, tls_jl = load_matrix("timing_jl.csv")
ms_py, ns_py, tqs_py, tls_py = load_matrix("timing_py.csv")

@assert ms_jl == ms_py
@assert ns_jl == ns_py
ms, ns = ms_jl, ns_jl
ms1 = fill(1, length(ms))
ns1 = fill(1, length(ns))


# High performance computing on your laptop II: algorithms, memory, and parallelism

![machine](figures/rube-goldberg.png)

Timothy E. Holy

Washington University in St. Louis

# Algorithms: the other half of performance

A fast language can be beat by a slow one *if* the code in the slow language solves the problem more efficiently.

*However*, in a slow language the converse can also be true: a poor algorithm might run faster than a good algorithm due to language limitations: certain constructs are implemented efficiently, and certain other constructs are not.

Julia's advantage: no language limitations! So use the best algorithm and you will get the best performance. Good for performance & *good for learning*.

# A simple case study: counting the number of times each value appears

In [ ]:
choices = 1:3
samples = rand(choices, 10)
print(samples)

How many 1s, 2s, and 3s are there in `samples`?

## Implementation #1: using `count`

To avoid measuring the time needed to allocate the output, let's pre-allocate a vector `counts` for the result:

```julia
function countelements_bilinear!(counts::AbstractVector{Int}, sequence)
    for idx in eachindex(counts)
        counts[idx] = count(==(idx), sequence)
    end
    return counts
end
```

How fast will this run? Let:
- `m = length(counts)`
- `n = length(sequence)`

We're calling `count` on `sequence` `m` times, so:

$$
t_\mathrm{total} = m t_\mathrm{count\ sequence}
$$

How is `count` implemented? Here's the (only lightly-edited) code (discover this with `@which`):

```julia
function count(pred, itr, init = 0)   # `pred` is short for "predicate function"
    n = init
    for x in itr
        n += pred(x)::Bool    # `pred(x)` must return `true` or `false` (error otherwise)
    end
    return n
end
```

So the running time $t_\mathrm{count\ sequence} \propto n$ (the number of items in `sequence`)

Thus, overall

$$
t_\mathrm{total} \propto m n
$$

This is *bilinear*, meaning linear in each of two variables, $m$ and $n$.

Examples:

| m | n | ttotal |
| ---:| ---:| ---:|
| 3 | 3 | 9 |
| 3 | 1000 | 3,000 |
| 1000 | 1000 | 1,000,000|

# Implementation #2: using indexing

```julia
function countelements_linear!(counts::AbstractVector{Int}, sequence)
    fill!(counts, 0)          # initialize all to 0
    for item in sequence
        counts[item] += 1     # increment appropriate counter for each item
    end
    return counts
end
```

$$
t_\mathrm{total} = t_\mathrm{fill} + n t_\mathrm{increment}
$$

But $t_\mathrm{increment}$ is approximately constant (doesn't depend on `length(counts)`)

`fill!` sets each element of `counts` to 0, so $t_\mathrm{fill} \propto m$.

Total: $t_\mathrm{total} \propto m + n$

This is linear in each variable separately.

Examples:

| m | n | t (calling `count`) | t (using indexing) |
| ---:| ---:| ---:| --:|
| 3 | 3 | 9 | 6 |
| 3 | 1000 | 3,000 | 1,003 |
| 1000 | 1000 | 1,000,000| 2,000 |

Use indexing for the win! The difference can be *orders of magnitude*.

# Visualize the running times for many different `m` & `n`

In [ ]:
plot3D(log10.(ms .* ns1')[:], log10.(ms1 .* ns')[:], log10.(tqs_jl)[:], ".", color="#8000FF")
plot3D(log10.(ms .* ns1')[:], log10.(ms1 .* ns')[:], log10.(tls_jl)[:], ".g")
legend(("julia-bilinear", "julia-linear"))
xlabel("log10(# choices)")
ylabel("log10(# samples)")
zlabel("log10(time)")

# Comparing Julia & Python

In [ ]:
plot3D(log10.(ms .* ns1')[:], log10.(ms1 .* ns')[:], log10.(tqs_jl)[:], ".", color="#8000FF")
plot3D(log10.(ms .* ns1')[:], log10.(ms1 .* ns')[:], log10.(tls_jl)[:], ".g")
plot3D(log10.(ms .* ns1')[:], log10.(ms1 .* ns')[:], log10.(tqs_py)[:], ".y")
plot3D(log10.(ms .* ns1')[:], log10.(ms1 .* ns')[:], log10.(tls_py)[:], ".b")
legend(("julia-bilinear", "julia-linear", "python-bilinear", "python-linear"))
xlabel("log10(# choices)")
ylabel("log10(# samples)")
zlabel("log10(time)")


In Python, `np.count` is written in C and is fast, but when we write the iterating & indexing code in Python, it's slow.

A *worse* algorithm gives *better* performance. Not just slow, but also confusing!

# Estimating performance: track the loops

```julia
for i = 1:m, j = 1:n
    # do something
end
```

Iterates through `n` items a total of `m` times, leading to $t \propto m n$.

## Example: matrix-vector multiplication

![matvec](figures/Matrix_vector_multiplication.png)

Function body (naive implementation):

```julia
for i = 1:m
    s = 0
    for j = 1:n
        s += M[i,j] * v[j]
    end
    out[i] = s
end
```

So the total time is approximately `m*n`.

# Big-O notation

You may notice there's a little work outside the innermost loop:

```julia
for i = 1:m
    s = 0                      # happens `m` times
    for j = 1:n
        s += M[i,j] * v[j]     # happens `m*n` times
    end
    out[i] = s                 # happens `m` times
end
```

More accurately, the running time is `a*m + b*m*n`, where `a` and `b` are constants.

*Big-O notation*: keep only the leading terms. `m*n` can be much bigger than `m`, so ignore the term proportional to `m`. We say this algorithm is $O(mn)$.

Good rule of thumb: keep all terms that *might* be leading.

Examples:

$t \propto an^2 + bn^3$: say it is $O(n^3)$

$t \propto an^2 + bmn + cm + dn$: say it is $O(n^2 + mn)$

# Analyzing an example from last session

```julia
function mult2(A, B, x)
#     C = A * B
#     return C * x
    y = B * x
    return A * y
end
```

For simplicity, let's use square matrices, `m = n`.

![matmatmul](figures/matmatmul.jpg)

Each entry of the output is $O(n)$.

There are $n^2$ such entries, so the total is $O(n^3)$.

So for our example:

```julia
function mult2(A, B, x)
#     C = A * B            # O(n^3)
#     return C * x         # O(n^2)
    y = B * x              # O(n^2)
    return A * y           # O(n^2)
end
```

$O(n^2)$ is better than $O(n^3)$, so the second implementation wins.

# A final example: binary search

"I'm thinking of a number between 1 and 100: guess which one it is!"

If I only tell you "right" or "wrong", you might have to try 100 guesses

If I tell you "smaller" or "bigger", you can get it in 7

Example:

| guess | answer | remaining |
| ---:|:--- |:--- |
| 50 | "smaller" | 1:49 |
|25 | "bigger" | 26:49 |
|37 | "smaller" | 26:36 |
|30 | "bigger" | 31:36 |
|33 | "smaller" | 31:32 |
|32 | "correct!" | |

Solving this problem is $O(\log n)$.

Lots of algorithms use such "divide and conquer" strategies and it frequently results in $O(\log n)$ cost.

# The difference between "not slow" and "fast"

Julia *almost* lets you compare algorithms in a "pure" sense

But even Julia is not immune to a few idiosyncracies of computer hardware. Avoiding idiosyncracies and leveraging opportunities specific to the hardware is the difference between "not slow" and "fast."

# A brief introduction to memory & cache

Accessing *most* memory is slow. It's possible, but expensive, to build very fast memory.

![cache1](figures/cache1.png)

Ulrich Drepper, [What every programmer should know about memory](https://people.freebsd.org/~lstewart/articles/cpumemory.pdf).

## Key rules about memory

- computers have multiple layers of cache
- the fastest caches have only ~1/10^6 of the storage of main memory
- main memory gets transferred to cache in small chunks of adjacent memory (*cache line*)
- performance is better if you exploit what you have in cache before flushing it and swapping in new memory
- CPUs try to predict what chunk will be needed next and start fetching it before you actually use it. Consequence: predictable access patterns are more efficient than unpredictable ones.

Tip: [LoopVectorization](https://github.com/JuliaSIMD/LoopVectorization.jl) will reorder your loops and make other changes that help "not slow" code become "fast."

# Parallelism

By default, code executes *serially* (in order). But increasingly, computers have multiple *cores* in the CPU:

![threading](figures/multithreading.png)

## The `Threads` standard library

Start Julia with multiple threads:

```sh
$ julia --threads 4
```


In [ ]:
Threads.nthreads()

In [ ]:
myid = Vector{Int}(undef, 8)
Threads.@threads for i = 1:8
    myid[i] = Threads.threadid()
end
myid'

## Know the costs & benefits of threading

A `Threads.@threads` has an overhead of a few microseconds, equivalent to thousands of computations. *Reserve threading for sizable jobs.*

In cases where threads can work independently, it's sometimes possible to get an `~n`-fold speedup from `n` threads.

Communicating between threads can be a source of bugs and/or slowdowns.

### Example: summation

In [ ]:
function threadedsum(v)
    s = 0
    Threads.@threads for i in eachindex(v)
        s = s + v[i]
    end
    return s
end

In [ ]:
v = rand(1:5, 10^4)
threadedsum(v)

In [ ]:
sum(v)

Problem: `s` from one thread is being overwritten by another thread

In [ ]:
### Summation with threading

function threadedsum2(v)
    spartial = zeros(Int, Threads.nthreads())
    Threads.@threads for i in eachindex(v)
        spartial[Threads.threadid()] += v[i]
    end
    return sum(spartial)
end

In [ ]:
threadedsum2(v)

In [ ]:
sum(v)

# Other opportunities in parallelism

- GPU computing: Julia can generate native code for GPUs! See [JuliaGPU](https://juliagpu.org/)
- Distributed computing: clusters

Caution: in slow languages, sometimes people reach almost unthinkingly for parallelism

One of Julia's advantages: often, you *don't need parallelism* to get the speed you need

Optimize a simple implementation first, and only then adopt more complex measures

# Course summary

Hopefully you've learned a lot!

Process:

- Open source, git, & GitHub
- Test-driven or test-centric development
- Continuous integration, documentation, etc.

High performance computing:

- Using Julia to escape the limitations of older, slower languages
- Using Julia well so that you don't inadvertently handicap your code
- Exploiting Julia's strengths to choose the best algorithm and opportunities to get the most out of your hardware

# Next steps

- diving deeper into algorithms & data structures: hash tables, heaps, priority queues, and many others
- extending your knowledge of "idiomatic" Julia: read package code and Julia's `Base` module
- participating in package development with more experienced Julia developers
- learning more about threading & high-performance computing

Good luck!